# Modelación y Simulación - Laboratorio 6
## Simulación de Eventos Discretos: Servidores

---
### Integrantes:
- Gustavo Andrés González 21438
- Diego Alberto Leiva 21752
- José Pablo Orellana 21970
- - -

## Librerias

In [1]:
import simpy
import random
import numpy as np

## Parametros y configuraciones

In [2]:
# Parámetros generales
LAMBDA = 2400 / 60  # Solicitudes por segundo
SIMULATION_TIME = 3600  # Tiempo de simulación en segundos (1 hora)
SEED = 42  # Semilla para reproducibilidad

# Configuración de los servidores
SERVIDOR_MMC = 1  # Mountain Mega Computing tiene un solo servidor
CAPACIDAD_MMC = 100  # Capacidad de Mountain Mega (solicitudes por segundo)

SERVIDORES_PIZZITA = 10  # 10 servidores para Pizzita computing
CAPACIDAD_PIZZITA = CAPACIDAD_MMC / 10  # Cada servidor de Pizzita tiene 1/10 de la capacidad de MMC

# Variables globales para la recolección de datos
class Metrics:
    def __init__(self):
        self.num_requests = 0
        self.total_waiting_time = 0
        self.queue_length = 0
        self.busy_time = 0
        self.idle_time = 0
        self.last_exit_time = 0

## Simulaciones

In [3]:
def handle_request(env, server, capacity, metrics):
    """
    Manejo de una solicitud en el servidor.
    """
    arrival_time = env.now
    metrics.num_requests += 1
    
    with server.request() as request:
        # Cola
        yield request
        waiting_time = env.now - arrival_time
        metrics.total_waiting_time += waiting_time
        
        # Tiempo de servicio
        service_time = random.expovariate(1 / capacity)
        yield env.timeout(service_time)
        
        metrics.busy_time += service_time
        metrics.last_exit_time = env.now

In [4]:
def arrival_process(env, server, capacity, metrics):
    """
    Proceso de llegada de solicitudes al sistema.

    Args:
        env: entorno de simulación
        server: servidor al que se enviarán las solicitudes
        capacity: capacidad del servidor
        metrics: objeto para recolectar métricas

    Yields:
        eventos de llegada de solicitudes
    """
    while True:
        # Generar nueva solicitud
        inter_arrival_time = random.expovariate(LAMBDA)
        yield env.timeout(inter_arrival_time)
        
        env.process(handle_request(env, server, capacity, metrics))

In [5]:

def run_simulation(num_servers, capacity, simulation_time=SIMULATION_TIME):
    """
    Función para correr la simulación de un sistema de colas.

    Args:
        num_servers: cantidad de servidores en el sistema
        capacity: capacidad de cada servidor
        simulation_time: tiempo de simulación

    Returns:
        diccionario con las métricas de la simulación
    """
    random.seed(SEED)
    env = simpy.Environment()
    
    server = simpy.Resource(env, capacity=num_servers)
    metrics = Metrics()
    
    env.process(arrival_process(env, server, capacity, metrics))
    
    env.run(until=simulation_time)
    
    metrics.idle_time = simulation_time * num_servers - metrics.busy_time
    avg_waiting_time = metrics.total_waiting_time / metrics.num_requests if metrics.num_requests > 0 else 0
    avg_queue_length = metrics.total_waiting_time / simulation_time
    
    return {
        "Solicitudes atendidas": metrics.num_requests,
        "Tiempo ocupado": metrics.busy_time,
        "Tiempo desocupado": metrics.idle_time,
        "Tiempo total en cola": metrics.total_waiting_time,
        "Tiempo promedio en cola": avg_waiting_time,
        "Promedio solicitudes en cola": avg_queue_length,
        "Última solicitud salida": metrics.last_exit_time
    }


## Resultados

In [6]:
def pretty_print(metrics):
    """
    Función para imprimir las métricas de la simulación de forma legible.

    Args:
        metrics: diccionario con las métricas de la simulación

    Returns:
        None
    """
    print("Métricas de la simulación:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")

In [7]:
# Simulación para Mountain Mega Computing
result_mmc = run_simulation(SERVIDOR_MMC, CAPACIDAD_MMC)
print("Mountain Mega Computing:")
pretty_print(result_mmc)

Mountain Mega Computing:
Métricas de la simulación:
Solicitudes atendidas: 143916.00
Tiempo ocupado: 3544.34
Tiempo desocupado: 55.66
Tiempo total en cola: 78898.24
Tiempo promedio en cola: 0.55
Promedio solicitudes en cola: 21.92
Última solicitud salida: 3544.37


In [8]:
# Simulación para Pizzita Computing
result_pizzita = run_simulation(SERVIDORES_PIZZITA, CAPACIDAD_PIZZITA)
print("\nPizzita Computing:")
pretty_print(result_pizzita)


Pizzita Computing:
Métricas de la simulación:
Solicitudes atendidas: 143949.00
Tiempo ocupado: 35923.21
Tiempo desocupado: 76.79
Tiempo total en cola: 6381590.65
Tiempo promedio en cola: 44.33
Promedio solicitudes en cola: 1772.66
Última solicitud salida: 3599.70
